Installazione dipendenze


In [ ]:
%pip install nltk
%pip install datasets
%pip install transformers[torch]
%pip install tokenizers
%pip install evaluate
%pip install rouge_score
%pip install sentencepiece
%pip install huggingface_hub
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Fine Tuning Flan T5 senza essere quantizzato

In [ ]:

from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Scegliamo il modello che vogliamo usare e carichiamo il tokenizer e il modello
MODEL_NAME = "google/flan-t5-large"
tokenizer = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
model = AutoTokenizer.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Path del training e del validation set
path_train = '/content/drive/MyDrive/Tesi/dataset/scierc_train_no_diff.json'
path_val = '/content/drive/MyDrive/Tesi/dataset/scierc_dev_no_diff.json'

# Carichiamo il dataset, train e validation
train = load_dataset('json', data_files=path_train, split='train')
val = load_dataset('json', data_files=path_val, split='train')


# Funzione di preprocessing

def preprocess_function(examples):
   # L'input è formato da prefix e dalla frase di input
   # Prefix
   prefix = "NER: "
   inputs = f"{prefix}: {examples['input']}"
   model_inputs = tokenizer(inputs, max_length=512, truncation=True)

   # Viene definito l'output
   keys = ["Metric", "Method", "Material", "Task", "OtherScientificTerm", "Generic"]
   text = ""
   for key in keys:
    for elem in examples[key]:
      text += f"""{key}: {elem}; """


   output = text[:-1]
   labels = tokenizer(text_target=output,
                      max_length=512,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

# Viene applicata la funzione di preprocessing ad entrambi i dataset
tokenized_dataset = train.map(preprocess_function)
tokenized_dataset2 = val.map(preprocess_function)

# Parametri globali
L_RATE = 3e-4
BATCH_SIZE = 16
PER_DEVICE_EVAL_BATCH = 8
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 2
NUM_EPOCHS = 15

output_dir = "/content/drive/MyDrive/Tesi/models/flan-t5-large-ner-no-diff-10"

# Prepariamo i parametri del training
training_args = Seq2SeqTrainingArguments(
   output_dir = output_dir,
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False,
   save_strategy="epoch",
)



trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset,
   eval_dataset=tokenized_dataset2,
   tokenizer=tokenizer,
   data_collator=data_collator,
)

# Avviamo il fine tuning

trainer.train()

Generazione risposte nel file JSON